# Automatic finding of thresholds of already calibrated data

In [ ]:
%matplotlib inline

import glob
import os
from collections.abc import Callable, Sequence, Iterator, Mapping
from abc import ABC, abstractmethod
from typing import Any

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes

import numpy as np
import awkward as ak

from lgdo import lh5
from lgdo.lh5.exceptions import LH5DecodeError
from legendmeta import LegendMetadata
from dspeed.processors import get_multi_local_extrema

from sipm_autocalib import *

plt.rcParams["figure.figsize"] = (10, 4)

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta  = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250807T150028Z")
chmap_sipm = chmap.map("system", unique=False).spms
#requires recent legend-datasets
raw_keys = chmap_sipm.map("analysis.usability", unique=False).on.map("daq.rawid").keys()

In [ ]:
raw_dir = os.path.join(proj_dir, "data/tier/raw/phy/p15/r004_part")
dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p15r004dsp_part")
orig_dsp_dir = os.path.join(proj_dir, "data/tier/dsp/phy/p15/r004")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()
def gimme_raw_filename_from_dsp(dspfilename: str):
    return dspfilename.replace(dsp_dir, raw_dir).replace("tier_dsp", "tier_raw")
def gimme_orig_dsp_filename(dspfilename: str):
    # get the original dsp files so I get pulser info
    return dspfilename.replace(dsp_dir, orig_dsp_dir)

In [ ]:
energies = get_energies(dsp_files, raw_keys, chmap, orig_dsp_file=[gimme_orig_dsp_filename(dsp) for dsp in dsp_files])

In [ ]:
calib, _ = read_override_file("../config/l200-p15-r004-lar-T%-par_hit-overwrite.yaml")

In [ ]:
calibrated_histos = get_calibrated_histograms(energies, calib, (0,5), 200)
thresholds = multi_valley_minima(calibrated_histos, mimimum_position=0.4)
_ = plot_all_pe_histograms_and_thresholds(calibrated_histos, thresholds, gridx=True)

In [ ]:
if True:
    output_override_file("../config/temp_overridex.yaml", calib, thresholds)